# 相邻区间站判断——数据筛选

> 关键点：去判断数据是否为两个相邻站点的余票

几部走：<br>
1. 获取沪宁线所有车次的区间站列表信息。
  - 获取车次编号；
  - 按车次编号获取停靠站点，并生成列表。

2. 区间站列表与余票数据中from_to进行比对，并最终获取有用信息

# 1. 区间站列表信息获取

## 1.1 获取沪宁线车次

In [79]:
import requests
import pandas as pd
import json
#import arrow
import warnings
warnings.filterwarnings("ignore")

def get_pd(para):
    para = para
    r = requests.get('https://kyfw.12306.cn/otn/lcxxcx/query?purpose_codes=ADULT&' + \
                     para, verify=False).text
    return r

def to_csv(js):
    js2 = json.loads(js)
    data = js2['data']['datas']
    df = pd.DataFrame(data)
    df_use = df[[u'train_no',
            u'station_train_code',
                 u'from_station_name',
                 u'to_station_name',
                 u'start_time',
                 u'swz_num',
                 u'tz_num',
                 u'zy_num',
                 u'ze_num',
                 u'wz_num',
                 ]]
    df_use = df_use[df_use[u'station_train_code'].str.startswith('G') | \
    df_use[u'station_train_code'].str.startswith('D')]
    df_use.replace('--',0,inplace=True)
    df_use.replace(u'无',0,inplace=True)
    df_use[u'train_no'].to_csv('./data/SH-NJ-train-no.csv',index=False,encoding='utf-8')
#     df_use[u'train_no'].to_csv('./data/NJ-SH-train_no.csv',index=False,encoding='utf-8')
    
if __name__ == '__main__':
    para = 'queryDate=2016-12-25&from_station=SHH&to_station=NJH'
#     para = 'queryDate=2016-12-29&from_station=NJH&to_station=SHH'
    js = get_pd(para)
    to_csv(js)
    print('finished')

finished


## 1.2 获取各车次，站点信息

https://kyfw.12306.cn/otn/czxx/queryByTrainNo?train_no=5l000G191200&from_station_telecode=SHH&to_station_telecode=NJH&depart_date=2017-01-06

In [98]:
import requests
import pandas as pd
import json
#import arrow
import warnings
warnings.filterwarnings("ignore")


def get_pd(para):
    para = para
    r = requests.get('https://kyfw.12306.cn/otn/czxx/queryByTrainNo?train_no='+para+\
                     '&from_station_telecode=SHH&to_station_telecode=NJH&depart_date=2016-12-25',
                     verify=False).text
    return r

def to_csv(para,js):
    js2 = json.loads(js)
    data = js2['data']['data']
    df = pd.DataFrame(data)
    try:
        df = df[["station_name",]]
        df_use = pd.DataFrame({'train_no':[''],'station_name':['']})
        df_use['train_no'] = para.strip()
        df_use['station_name'] = df_use['station_name'].apply(lambda x: df["station_name"].values)
        df_use.to_csv('./data/SH-NJ-station.csv',mode = 'a',index=False,header=False,encoding='utf-8')
    except:
        pass
if __name__ == '__main__':
    station_no_file = open('./data/SH-NJ-train-no.csv','r')
    station_no_data = station_no_file.readlines()
    for station_no in station_no_data:
        js = get_pd(station_no.strip())
        to_csv(station_no.strip(),js)
    print('finished')

finished


>可用数据：<br>
>`./data/SH-NJ-train-no.csv`

### 读取信息

In [179]:
df_SH_NJ = pd.read_csv('./data/SH-NJ-station.csv', encoding='utf-8')
df_SH_NJ.tail()

,"[上海, 无锡, 常州, 丹阳, 镇江, 宝华山, 南京南, 马鞍山东, 当涂东, 芜湖, 弋江, 铜陵, 池州, 安庆]",55000G707270
201,"[上海虹桥, 昆山南, 苏州, 无锡, 常州, 镇江, 南京]",5l000G7128A0
202,"[深圳北, 惠州南, 惠东, 汕尾, 潮阳, 潮汕, 诏安, 漳浦, 漳州, 厦门北, 泉州...",6i000D228207
203,"[苍南, 鳌江, 温州南, 青田, 丽水, 缙云西, 永康南, 武义北, 义乌, 杭州东, ...",5j000G735010
204,"[上海, 昆山南, 苏州, 无锡, 常州, 南京]",55000G7098B0
205,"[上海, 苏州, 南京, 徐州, 郑州东, 洛阳龙门, 三门峡南, 华山北, 渭南北, 西安北]",550000D30661


In [180]:
df_SH_NJ = pd.read_csv('./data/SH-NJ-station.csv',names=['station_name','train_no'], encoding='utf-8')
df_SH_NJ.tail()

,station_name,train_no
202,"[上海虹桥, 昆山南, 苏州, 无锡, 常州, 镇江, 南京]",5l000G7128A0
203,"[深圳北, 惠州南, 惠东, 汕尾, 潮阳, 潮汕, 诏安, 漳浦, 漳州, 厦门北, 泉州...",6i000D228207
204,"[苍南, 鳌江, 温州南, 青田, 丽水, 缙云西, 永康南, 武义北, 义乌, 杭州东, ...",5j000G735010
205,"[上海, 昆山南, 苏州, 无锡, 常州, 南京]",55000G7098B0
206,"[上海, 苏州, 南京, 徐州, 郑州东, 洛阳龙门, 三门峡南, 华山北, 渭南北, 西安北]",550000D30661


In [181]:
df_SH_NJ[df_SH_NJ['train_no'] == '5l000G7128A0']

,station_name,train_no
202,"[上海虹桥, 昆山南, 苏州, 无锡, 常州, 镇江, 南京]",5l000G7128A0


In [182]:
type(df_SH_NJ.iloc[201,0])

unicode

### 添加列，进行组合

In [144]:
df_SH_NJ['station_name'] = df_SH_NJ['station_name'].apply(lambda x: \
                                              x.replace(u'[',u'').replace(u']',u'').replace(' ','').split(','))

In [145]:
df_SH_NJ.iloc[201,0]

[u'\u4e0a\u6d77', u'\u82cf\u5dde', u'\u5357\u4eac', u'\u5317\u4eac\u5357']

In [146]:
type(df_SH_NJ.iloc[201,0])

list

### 区间站组合

#### 示例：将2数进行组合（zip与map函数、join方法的用法）

In [ ]:
seq1 = ['one0', 'one1', 'one2']
seq2 = ['two0', 'two1', 'two2']
zip(seq1, seq2)

`['A0','A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7']`<br>

`['A0', 'A1', 'A2', 'A3']`<br>
`['A4', 'A5', 'A6', 'A7']`<br>

In [22]:
seq = ['A0','A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7']
com= map(list,zip(seq[:-1], seq[1:]))
com

[['A0', 'A1'],
 ['A1', 'A2'],
 ['A2', 'A3'],
 ['A3', 'A4'],
 ['A4', 'A5'],
 ['A5', 'A6'],
 ['A6', 'A7']]

In [147]:
df_stations = df_SH_NJ.copy()

In [183]:
df_stations['station_interval'] = df_stations['station_name'].apply(lambda x: map(list, zip(x[:-1], x[1:])))
df_stations.tail()

,station_name,train_no,station_interval
202,"[上海虹桥, 昆山南, 苏州, 无锡, 常州, 镇江, 南京]",5l000G7128A0,"[[上海虹桥, 昆山南], [昆山南, 苏州], [苏州, 无锡], [无锡, 常州], [..."
203,"[深圳北, 惠州南, 惠东, 汕尾, 潮阳, 潮汕, 诏安, 漳浦, 漳州, 厦门北, 泉州...",6i000D228207,"[[深圳北, 惠州南], [惠州南, 惠东], [惠东, 汕尾], [汕尾, 潮阳], [潮..."
204,"[苍南, 鳌江, 温州南, 青田, 丽水, 缙云西, 永康南, 武义北, 义乌, 杭州东, ...",5j000G735010,"[[苍南, 鳌江], [鳌江, 温州南], [温州南, 青田], [青田, 丽水], [丽水..."
205,"[上海, 昆山南, 苏州, 无锡, 常州, 南京]",55000G7098B0,"[[上海, 昆山南], [昆山南, 苏州], [苏州, 无锡], [无锡, 常州], [常州..."
206,"[上海, 苏州, 南京, 徐州, 郑州东, 洛阳龙门, 三门峡南, 华山北, 渭南北, 西安北]",550000D30661,"[[上海, 苏州], [苏州, 南京], [南京, 徐州], [徐州, 郑州东], [郑州东..."


In [184]:
df_stations.station_interval[0]

[[u'\u4e0a\u6d77', u'\u65e0\u9521'],
 [u'\u65e0\u9521', u'\u5e38\u5dde'],
 [u'\u5e38\u5dde', u'\u4e39\u9633'],
 [u'\u4e39\u9633', u'\u9547\u6c5f'],
 [u'\u9547\u6c5f', u'\u5b9d\u534e\u5c71'],
 [u'\u5b9d\u534e\u5c71', u'\u5357\u4eac\u5357'],
 [u'\u5357\u4eac\u5357', u'\u9a6c\u978d\u5c71\u4e1c'],
 [u'\u9a6c\u978d\u5c71\u4e1c', u'\u5f53\u6d82\u4e1c'],
 [u'\u5f53\u6d82\u4e1c', u'\u829c\u6e56'],
 [u'\u829c\u6e56', u'\u5f0b\u6c5f'],
 [u'\u5f0b\u6c5f', u'\u94dc\u9675'],
 [u'\u94dc\u9675', u'\u6c60\u5dde'],
 [u'\u6c60\u5dde', u'\u5b89\u5e86']]

In [185]:
type(df_stations['station_interval'][0][0])

list

# 2、区间站列表与余票数据中from_to进行比对，并最终获取有用信息
余票数据与站点区间数据比对，获取是相邻站点的余票数据信息

In [186]:
import pandas as pd
import numpy as np
df_tickets = pd.read_json('./data/hu_ning_ft1.json')

In [187]:
len(df_tickets)

2814

In [188]:
print df_stations['station_interval'][0]

[[u'\u4e0a\u6d77', u'\u65e0\u9521'], [u'\u65e0\u9521', u'\u5e38\u5dde'], [u'\u5e38\u5dde', u'\u4e39\u9633'], [u'\u4e39\u9633', u'\u9547\u6c5f'], [u'\u9547\u6c5f', u'\u5b9d\u534e\u5c71'], [u'\u5b9d\u534e\u5c71', u'\u5357\u4eac\u5357'], [u'\u5357\u4eac\u5357', u'\u9a6c\u978d\u5c71\u4e1c'], [u'\u9a6c\u978d\u5c71\u4e1c', u'\u5f53\u6d82\u4e1c'], [u'\u5f53\u6d82\u4e1c', u'\u829c\u6e56'], [u'\u829c\u6e56', u'\u5f0b\u6c5f'], [u'\u5f0b\u6c5f', u'\u94dc\u9675'], [u'\u94dc\u9675', u'\u6c60\u5dde'], [u'\u6c60\u5dde', u'\u5b89\u5e86']]


In [189]:
df_tickets[df_tickets['station_train_code'] == 'G7128'].head(1)

,from_station_name,from_to,record_date,record_time,start_time,start_time_before30m,station_train_code,swz_num,to_station_name,train_no,tz_num,wz_num,ze_num,zy_num
2401712,上海虹桥,"[上海虹桥, 昆山南]",2016-12-21,2016-12-24 20:40:00,2016-12-24 21:10:00,20:40,G7128,0,昆山南,5l000G7128A0,0,168,309,29


In [190]:
df_tickets_com = pd.merge(df_tickets,df_stations,on='train_no')
df_tickets_com[df_tickets_com['station_train_code'] == 'G7128']

,from_station_name,from_to,record_date,record_time,start_time,start_time_before30m,station_train_code,swz_num,to_station_name,train_no,tz_num,wz_num,ze_num,zy_num,station_name,station_interval
2733,上海虹桥,"[上海虹桥, 昆山南]",2016-12-21,2016-12-24 20:40:00,2016-12-24 21:10:00,20:40,G7128,0,昆山南,5l000G7128A0,0,168,309,29,"[上海虹桥, 昆山南, 苏州, 无锡, 常州, 镇江, 南京]","[[上海虹桥, 昆山南], [昆山南, 苏州], [苏州, 无锡], [无锡, 常州], [..."
2734,上海虹桥,"[上海虹桥, 苏州]",2016-12-21,2016-12-24 20:40:00,2016-12-24 21:10:00,20:40,G7128,0,苏州,5l000G7128A0,0,168,255,27,"[上海虹桥, 昆山南, 苏州, 无锡, 常州, 镇江, 南京]","[[上海虹桥, 昆山南], [昆山南, 苏州], [苏州, 无锡], [无锡, 常州], [..."
2735,上海虹桥,"[上海虹桥, 无锡]",2016-12-21,2016-12-24 20:40:00,2016-12-24 21:10:00,20:40,G7128,0,无锡,5l000G7128A0,0,168,282,25,"[上海虹桥, 昆山南, 苏州, 无锡, 常州, 镇江, 南京]","[[上海虹桥, 昆山南], [昆山南, 苏州], [苏州, 无锡], [无锡, 常州], [..."
2736,上海虹桥,"[上海虹桥, 常州]",2016-12-21,2016-12-24 20:40:00,2016-12-24 21:10:00,20:40,G7128,0,常州,5l000G7128A0,0,168,267,25,"[上海虹桥, 昆山南, 苏州, 无锡, 常州, 镇江, 南京]","[[上海虹桥, 昆山南], [昆山南, 苏州], [苏州, 无锡], [无锡, 常州], [..."
2737,上海虹桥,"[上海虹桥, 镇江]",2016-12-21,2016-12-24 20:40:00,2016-12-24 21:10:00,20:40,G7128,0,镇江,5l000G7128A0,0,168,245,25,"[上海虹桥, 昆山南, 苏州, 无锡, 常州, 镇江, 南京]","[[上海虹桥, 昆山南], [昆山南, 苏州], [苏州, 无锡], [无锡, 常州], [..."
2738,上海虹桥,"[上海虹桥, 南京]",2016-12-21,2016-12-24 20:40:00,2016-12-24 21:10:00,20:40,G7128,0,南京,5l000G7128A0,0,168,266,24,"[上海虹桥, 昆山南, 苏州, 无锡, 常州, 镇江, 南京]","[[上海虹桥, 昆山南], [昆山南, 苏州], [苏州, 无锡], [无锡, 常州], [..."
2739,昆山南,"[昆山南, 苏州]",2016-12-21,2016-12-24 21:01:00,2016-12-24 21:31:00,21:01,G7128,0,苏州,5l000G7128A0,0,168,196,22,"[上海虹桥, 昆山南, 苏州, 无锡, 常州, 镇江, 南京]","[[上海虹桥, 昆山南], [昆山南, 苏州], [苏州, 无锡], [无锡, 常州], [..."
2740,昆山南,"[昆山南, 无锡]",2016-12-21,2016-12-24 21:01:00,2016-12-24 21:31:00,21:01,G7128,0,无锡,5l000G7128A0,0,168,190,20,"[上海虹桥, 昆山南, 苏州, 无锡, 常州, 镇江, 南京]","[[上海虹桥, 昆山南], [昆山南, 苏州], [苏州, 无锡], [无锡, 常州], [..."
2741,昆山南,"[昆山南, 常州]",2016-12-21,2016-12-24 21:01:00,2016-12-24 21:31:00,21:01,G7128,0,常州,5l000G7128A0,0,168,178,20,"[上海虹桥, 昆山南, 苏州, 无锡, 常州, 镇江, 南京]","[[上海虹桥, 昆山南], [昆山南, 苏州], [苏州, 无锡], [无锡, 常州], [..."
2742,昆山南,"[昆山南, 镇江]",2016-12-21,2016-12-24 21:01:00,2016-12-24 21:31:00,21:01,G7128,0,镇江,5l000G7128A0,0,168,190,21,"[上海虹桥, 昆山南, 苏州, 无锡, 常州, 镇江, 南京]","[[上海虹桥, 昆山南], [昆山南, 苏州], [苏州, 无锡], [无锡, 常州], [..."


```
print (my_df.apply(lambda x: x.A in x.B, axis=1))
0     True
1    False
2     True
dtype: bool
print (my_df[my_df.apply(lambda x: x.A in x.B, axis=1)])
          A                               B
0  [aa, ab]  [[aa, ab], [ba, bb], [bc, bd]]
2  [bc, bd]            [[aa, ab], [bc, bd]]
```

In [191]:
df_tickets_com2 = df_tickets_com[df_tickets_com.apply(lambda x: x['from_to'] in x['station_interval'], axis=1)]
df_tickets_com2.tail(2)

,from_station_name,from_to,record_date,record_time,start_time,start_time_before30m,station_train_code,swz_num,to_station_name,train_no,tz_num,wz_num,ze_num,zy_num,station_name,station_interval
2771,上海,"[上海, 苏州]",2016-12-21,2016-12-24 21:54:00,2016-12-24 22:24:00,21:54,D306,0,苏州,550000D30661,0,0,77,0,"[上海, 苏州, 南京, 徐州, 郑州东, 洛阳龙门, 三门峡南, 华山北, 渭南北, 西安北]","[[上海, 苏州], [苏州, 南京], [南京, 徐州], [徐州, 郑州东], [郑州东..."
2773,苏州,"[苏州, 南京]",2016-12-21,2016-12-24 22:44:00,2016-12-24 23:14:00,22:44,D306,0,南京,550000D30661,0,0,4,0,"[上海, 苏州, 南京, 徐州, 郑州东, 洛阳龙门, 三门峡南, 华山北, 渭南北, 西安北]","[[上海, 苏州], [苏州, 南京], [南京, 徐州], [徐州, 郑州东], [郑州东..."


In [193]:
df_tickets_com2[df_tickets_com2['station_train_code'] == 'G7128'].head(2)

,from_station_name,from_to,record_date,record_time,start_time,start_time_before30m,station_train_code,swz_num,to_station_name,train_no,tz_num,wz_num,ze_num,zy_num,station_name,station_interval
2733,上海虹桥,"[上海虹桥, 昆山南]",2016-12-21,2016-12-24 20:40:00,2016-12-24 21:10:00,20:40,G7128,0,昆山南,5l000G7128A0,0,168,309,29,"[上海虹桥, 昆山南, 苏州, 无锡, 常州, 镇江, 南京]","[[上海虹桥, 昆山南], [昆山南, 苏州], [苏州, 无锡], [无锡, 常州], [..."
2739,昆山南,"[昆山南, 苏州]",2016-12-21,2016-12-24 21:01:00,2016-12-24 21:31:00,21:01,G7128,0,苏州,5l000G7128A0,0,168,196,22,"[上海虹桥, 昆山南, 苏州, 无锡, 常州, 镇江, 南京]","[[上海虹桥, 昆山南], [昆山南, 苏州], [苏州, 无锡], [无锡, 常州], [..."


In [194]:
len(df_tickets_com2)

851

In [199]:
df_tickets_com2.to_json('./data/tickets_to_analysi.json')